In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-3.0466e-06), tensor(1.))

review the f"....." python thing; 

In [7]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [10]:
# num hidden
nh = 50

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [11]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [105]:
def lin(x, w, b): return x@w + b

In [109]:
t = lin(x_valid, w1, b1)

In [110]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0138), tensor(0.9734))

In [102]:
def relu(x): return x.clamp_min(0.) # use functions written in pytorch as they are in 
# c speed in general

In [111]:
t = relu(lin(x_valid, w1, b1))

In [112]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3907), tensor(0.5836))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [14]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [15]:
w1.mean(),w1.std()

(tensor(0.0003), tensor(0.0505))

In [115]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.6079), tensor(0.8885))

In [16]:
#export
from torch.nn import init

in pytorch, F.linear always means pytorch.nn.functional

In [90]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') #fan out because pytorch lin matmul w/ transpose
t = relu(lin(x_valid, w1, b1))

In [27]:
init.kaiming_normal_??

In [91]:
w1.mean(),w1.std()

(tensor(-8.7407e-05), tensor(0.0505))

In [92]:
t.mean(),t.std()

(tensor(0.6124), tensor(0.8561))

In [93]:
w1.shape

torch.Size([784, 50])

In [94]:
import torch.nn

In [95]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [32]:
torch.nn.Linear.forward??

In [39]:
torch.nn.functional.linear??

In [40]:
torch.nn.Conv2d??

In [42]:
torch.nn.modules.conv._ConvNd.reset_parameters??  #init with math.sqrt(5) doesn't work well in general

SyntaxError: invalid syntax (<ipython-input-42-10987a915be9>, line 1)

In [17]:
# what if...? new relu to improve help maintain mean after relu layer
def relu(x): return x.clamp_min(0.) - 0.5

In [118]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0694), tensor(0.8233))

In [119]:
def model(xb): #in pytorch, a model can be a function; xb means a batch of x
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [120]:
%timeit -n 10 _=model(x_valid)

2.82 ms ± 710 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
x_valid.shape

torch.Size([10000, 784])

In [121]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [51]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [18]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

squeeze() without arguments will squeeze all unit axis. might be an issue for batch size of 1; So be specific here. 

In [19]:
y_train,y_valid = y_train.float(),y_valid.float() #make sure numbers are floats for MSE

In [54]:
preds = model(x_train)

In [55]:
preds.shape

torch.Size([50000, 1])

In [122]:
mse(preds, y_train)

tensor(28.4684)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [86]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [87]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [88]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [89]:
w1.shape, b1.shape, w2.shape, b2.shape

(torch.Size([784, 50]), torch.Size([50]), torch.Size([50, 1]), torch.Size([1]))

In [90]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2 # w2 with shape 50,1 due to using MSE, correctly should use crossentropy loss;
    # we don't actually need the loss in backward!
    loss = mse(out, targ)  # loss doesn't appear in grad, but loss' does!
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [91]:
forward_and_backward(x_train, y_train)

In [92]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [93]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [94]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [95]:
loss = forward(xt2, y_train)

In [96]:
loss.backward()

In [97]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [35]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [39]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b  #init includes the trainable parameters
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        # read more about the below two lines
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [41]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [43]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)  #interesting setup
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()   #doesn't this line computed all gradients?? pytorch
        for l in reversed(self.layers): l.backward() #this line computes gradients in own code;

In [40]:
x_train.shape, y_train.shape

(torch.Size([50000, 784]), torch.Size([50000]))

In [44]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [45]:
%time loss = model(x_train, y_train)

CPU times: user 120 ms, sys: 4 ms, total: 124 ms
Wall time: 15.6 ms


In [46]:
%time model.backward()

CPU times: user 4.8 s, sys: 5.95 s, total: 10.8 s
Wall time: 1.44 s


In [47]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [48]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [49]:
test = Module()

In [51]:
test()

Exception: not implemented

In [52]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [53]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [54]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [55]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [56]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [58]:
%time loss = model(x_train, y_train)

CPU times: user 112 ms, sys: 12 ms, total: 124 ms
Wall time: 15.8 ms


In [59]:
%time model.backward()

CPU times: user 364 ms, sys: 148 ms, total: 512 ms
Wall time: 64.2 ms


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [64]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [65]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [66]:
%time loss = model(x_train, y_train)

CPU times: user 152 ms, sys: 12 ms, total: 164 ms
Wall time: 20.3 ms


In [67]:
%time model.backward()

CPU times: user 380 ms, sys: 188 ms, total: 568 ms
Wall time: 71.2 ms


In [68]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [69]:
#export
from torch import nn

In [108]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [71]:
m

784

In [109]:
model = Model(m, nh, 1)

In [ ]:
%time loss = model(x_train, y_train)

In [100]:
%time loss = model(xt2, y_train)

CPU times: user 136 ms, sys: 20 ms, total: 156 ms
Wall time: 19.9 ms


In [101]:
%time loss.backward()

CPU times: user 376 ms, sys: 128 ms, total: 504 ms
Wall time: 63.4 ms


In [103]:
loss

tensor(28.5610, grad_fn=<MeanBackward0>)

In [102]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

AssertionError: near:
tensor([[ 7.1437e-06,  3.4914e-06, -9.1607e-06,  ...,  9.0436e-06,
         -8.4337e-06,  5.9365e-06],
        [-2.0114e-07, -3.1021e-07, -9.9879e-07,  ...,  1.0750e-07,
         -2.6583e-07, -8.2865e-07],
        [ 3.5673e-07, -1.8353e-06, -1.6877e-05,  ..., -2.6569e-06,
         -4.9048e-06, -1.8052e-06],
        ...,
        [-9.2422e-06, -2.1184e-06, -2.1862e-05,  ...,  1.5915e-06,
         -5.7936e-06,  4.2823e-06],
        [-6.6150e-06, -4.6958e-07, -1.4193e-05,  ...,  1.1209e-06,
         -7.2693e-06, -1.7211e-06],
        [-4.2026e-07,  8.1607e-06, -1.8777e-05,  ...,  8.0199e-06,
         -1.3776e-05,  1.1619e-05]])
tensor([[ 8.4960e-06,  4.5412e-06, -8.1657e-06,  ...,  7.5796e-06,
         -5.0369e-06,  5.7788e-06],
        [-2.6533e-07, -3.2504e-07, -1.0826e-06,  ...,  1.6552e-07,
         -3.6394e-07, -8.6939e-07],
        [ 2.4710e-06, -1.6422e-06, -1.7044e-05,  ..., -3.8739e-06,
         -3.7975e-06, -1.3298e-06],
        ...,
        [-5.8451e-06, -5

In [104]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = [nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,1)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [105]:
model = Model()

In [106]:
%time loss = model(x_train, y_train)

CPU times: user 128 ms, sys: 36 ms, total: 164 ms
Wall time: 20.4 ms


In [107]:
loss

tensor(25.7896, grad_fn=<MeanBackward0>)

In [81]:
%time loss = model(xt2, y_train)

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 13.7 ms


In [82]:
%time loss.backward()

CPU times: user 248 ms, sys: 136 ms, total: 384 ms
Wall time: 48.5 ms


In [85]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

TypeError: allclose(): argument 'input' (position 1) must be Tensor, not NoneType

## Export

In [ ]:
!./notebook2script.py 02_fully_connected-Copy1.ipynb

Converted 02_fully_connected.ipynb to nb_02.py
